In [2]:
# setting the environment variables
import sys
import os

sys.path.insert(0, os.path.abspath('..'))

from config import set_environment
set_environment()

## ConversationBufferMemory

In [3]:
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
  model_name="gpt-3.5-turbo", temperature=0, streaming=True
)

# Creating a conversation chain with memory
memory = ConversationBufferMemory()
chain = ConversationChain(llm=llm, memory=memory)
# User inputs a message
user_input = "Hi, how are you?"
# Processing the user input in the conversation chain
response = chain.predict(input=user_input)
# Printing the response
print(response)
# User inputs another message
user_input = "What's the weather like today?"
# Processing the user input in the conversation chain
response = chain.predict(input=user_input)
# Printing the response
print(response)
# Printing the conversation history stored in memory
print(memory.chat_memory.messages)

Hello! I'm doing well, thank you for asking. I've been busy processing a lot of information and learning new things. How can I assist you today?
The weather today is partly cloudy with a high of 75 degrees Fahrenheit and a 20% chance of rain in the afternoon. The wind is coming from the northwest at 10 mph. Is there anything else you would like to know?
[HumanMessage(content='Hi, how are you?'), AIMessage(content="Hello! I'm doing well, thank you for asking. I've been busy processing a lot of information and learning new things. How can I assist you today?"), HumanMessage(content="What's the weather like today?"), AIMessage(content='The weather today is partly cloudy with a high of 75 degrees Fahrenheit and a 20% chance of rain in the afternoon. The wind is coming from the northwest at 10 mph. Is there anything else you would like to know?')]


## ConversationBufferWindowMemory

In [4]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})


In [5]:
memory.load_memory_variables({})

{'history': 'Human: not much you\nAI: not much'}

### Customizations

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(temperature=0)
template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI Assistant:"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory(ai_prefix="AI Assistant"),
)
conversation("hello")

/Users/ben/anaconda3/envs/newgen/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hello
AI Assistant:

> Finished chain.


{'input': 'hello',
 'history': '',
 'response': ' Hello! How can I assist you today?'}

In [8]:
conversation("What's your name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hello
AI Assistant:  Hello! How can I assist you today?
Human: What's your name?
AI Assistant:

> Finished chain.


{'input': "What's your name?",
 'history': 'Human: hello\nAI Assistant:  Hello! How can I assist you today?',
 'response': ' My name is AI Assistant. I am an artificial intelligence designed to assist and communicate with humans. I do not have a physical form, but I exist in the digital world. Is there anything else you would like to know about me?'}

## ConversationSummaryMemory

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import OpenAI

# Initialize the summary memory and the language model
memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
# Save the context of an interaction
memory.save_context({"input": "hi"}, {"output": "whats up"})
# Load the summarized memory
memory.load_memory_variables({})


## CombinedMemory

In [10]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import (
   ConversationBufferMemory, CombinedMemory, ConversationSummaryMemory
)

# Initialize language model (with desired temperature parameter)
llm = OpenAI(temperature=0)
# Define Conversation Buffer Memory (for retaining all past messages)
conv_memory = ConversationBufferMemory(memory_key="chat_history_lines", input_key="input")
# Define Conversation Summary Memory (for summarizing conversation)
summary_memory = ConversationSummaryMemory(llm=llm, input_key="input")
# Combine both memory types
memory = CombinedMemory(memories=[conv_memory, summary_memory])
# Define Prompt Template
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
Summary of conversation:
{history}
Current conversation:
{chat_history_lines}
Human: {input}
AI:"""
PROMPT = PromptTemplate(input_variables=["history", "input", "chat_history_lines"], template=_DEFAULT_TEMPLATE)
# Initialize the Conversation Chain
conversation = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=PROMPT)
# Start the conversation
conversation("Hi!")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
Summary of conversation:

Current conversation:

Human: Hi!
AI:

> Finished chain.


{'input': 'Hi!',
 'chat_history_lines': '',
 'history': '',
 'response': " Hello there! It's nice to meet you. My name is AI and I am an artificial intelligence designed to assist and communicate with humans. How can I help you today?"}

## Persistance

In [11]:
from langchain.memory import ZepMemory
ZEP_API_URL = "http://localhost:8000"
ZEP_API_KEY = "<your JWT token>"
session_id = str(uuid4())

memory = ZepMemory(
    session_id=session_id,
    url=ZEP_API_URL,
    api_key=ZEP_API_KEY,
    memory_key="chat_history",
)


NameError: name 'uuid4' is not defined